In [1]:
import torch 
import torch.nn as nn

In [2]:
GPT_CONFIG_124M = {
    "vocab_size":50257,
    "context_length":1024,
    "embedding_dim":768,
    "num_of_heads":12,
    "num_of_layers":12,
    "drop_rate":0.1,
    "qkv_bias":False
}

In [3]:

class MultiHeadAttention(nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,num_of_head,qkv_bias=False):
        super().__init__()
        
        assert (d_out % num_of_head == 0), "d_out must be divisible by num_of_head"
        
        self.d_out = d_out                                   
        self.dropout = nn.Dropout(dropout)
        self.num_of_head = num_of_head                         
        
        self.head_dim = d_out // num_of_head                
        self.out_proj = nn.Linear(d_out, d_out)             
        
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length,context_length), diagonal=1)
        )
        
    def forward(self, x):
        num_of_inputs, num_of_tokens, embedding_dim = x.shape           
        
        queries = self.W_query(x)                            
        keys = self.W_key(x)
        values = self.W_value(x)
        
        queries = queries.view(num_of_inputs, num_of_tokens, self.num_of_head, self.head_dim)      
        keys = keys.view(num_of_inputs, num_of_tokens, self.num_of_head, self.head_dim)       
        values = values.view(num_of_inputs, num_of_tokens, self.num_of_head, self.head_dim)       
        
        
        queries = queries.transpose(1,2)
        keys = keys.transpose(1,2)
        values = values.transpose(1,2)
        
        attention_score = queries @ keys.transpose(2,3)
        mask_bool = self.mask.bool() [:num_of_tokens, :num_of_tokens]
        
        attention_score.masked_fill_(mask_bool, -torch.inf)
        
        k_d = keys.shape[-1]
        attention_weight = torch.softmax(attention_score / k_d**0.5, dim=-1)
        attention_weight = self.dropout(attention_weight)
        
        context_vector = (attention_weight @ values).transpose(1,2)
        
        context_vector = context_vector.contiguous().view(
            num_of_inputs, num_of_tokens, self.d_out
        )
        context_vector = self.out_proj(context_vector)
        return context_vector

In [4]:
class LayerNormalization(nn.Module):
    def __init__(self, embedding_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(embedding_dim))
        self.shift = nn.Parameter(torch.zeros(embedding_dim))
        
    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        variance = x.var(dim=-1, keepdim=True)
        
        x_normalized = (x - mean)/ torch.sqrt(variance + self.eps)
        return self.scale * x_normalized + self.shift

In [5]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self,x):
        return 0.5 * x * (
            1 + torch.tanh(torch.sqrt(torch.tensor(2.0/torch.pi)) * (x + 0.44715 * torch.pow(x,3)))
        )

In [6]:
class FeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(config['embedding_dim'], 4* config['embedding_dim']),
            GELU(),
            nn.Linear(4*config['embedding_dim'],config['embedding_dim'])
        )
        
    def forward(self, x):
        return self.layers(x)

In [7]:
inputs = torch.rand(2,4,768)

In [10]:
class TransformerBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layer_norm1 = LayerNormalization(config['embedding_dim'])
        self.attention = MultiHeadAttention(
            d_in=config['embedding_dim'],
            d_out=config['embedding_dim'],
            context_length=config['context_length'],
            dropout=config['drop_rate'],
            num_of_head=config['num_of_heads'],
            qkv_bias=config['qkv_bias']
        )
        self.dropout = nn.Dropout(config['drop_rate'])
        self.layer_norm2 = LayerNormalization(config['embedding_dim'])     
        self.ff = FeedForward(config)
        
    def forward(self,x):
        shortcut = x
        
        x = self.layer_norm1(x)
        x = self.attention(x)
        x = self.dropout(x)
        
        x = x + shortcut
        
        shortcut = x 
        x = self.layer_norm2(x)
        x = self.ff(x)
        x = self.dropout(x)
        
        x = x + shortcut
        return x

In [11]:
transformerBlock = TransformerBlock(GPT_CONFIG_124M) 

In [12]:
transformerBlock

TransformerBlock(
  (layer_norm1): LayerNormalization()
  (attention): MultiHeadAttention(
    (dropout): Dropout(p=0.1, inplace=False)
    (out_proj): Linear(in_features=768, out_features=768, bias=True)
    (W_query): Linear(in_features=768, out_features=768, bias=False)
    (W_key): Linear(in_features=768, out_features=768, bias=False)
    (W_value): Linear(in_features=768, out_features=768, bias=False)
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (layer_norm2): LayerNormalization()
  (ff): FeedForward(
    (layers): Sequential(
      (0): Linear(in_features=768, out_features=3072, bias=True)
      (1): GELU()
      (2): Linear(in_features=3072, out_features=768, bias=True)
    )
  )
)

In [14]:
output = transformerBlock(inputs)
output

tensor([[[ 0.3793,  0.7838,  0.0962,  ...,  0.4302, -0.0274,  0.3041],
         [ 0.3256,  0.9050,  1.2575,  ...,  0.2170,  0.5528, -0.1441],
         [ 0.8646,  0.0507,  0.4706,  ...,  0.1126,  0.7288, -0.0499],
         [ 0.4700, -0.0883,  0.6794,  ..., -0.4320,  0.5718,  0.5416]],

        [[ 0.1838, -0.2686,  0.3959,  ...,  1.0510,  0.2919,  0.1440],
         [ 0.7008,  0.3540,  0.5134,  ...,  0.8444,  1.1692,  0.6838],
         [ 0.7357,  0.3395,  0.2273,  ...,  1.6239,  0.7187,  0.7488],
         [ 0.5821,  0.5261,  0.3100,  ...,  1.5258,  0.2266,  0.4489]]],
       grad_fn=<AddBackward0>)

In [16]:
print(inputs.shape, output.shape)

torch.Size([2, 4, 768]) torch.Size([2, 4, 768])
